In [ ]:
# num_sample_vary.py

import numpy as np
from datetime import datetime

from SQDMetal.COMSOL.Model import COMSOL_Model
from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
from adjoint_sim_sf.AdjointSolver import AdjointEvaluator
from adjoint_sim_sf.Optimiser import Optimiser
from adjoint_sim_sf import Experiment

# 1) COMSOL engine
if COMSOL_Model._engine is None:
    COMSOL_Model.init_engine()

# 2) Evaluator config (must match AdjointEvaluator.update_params / attributes)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 20,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

# 3) Objects
designer  = SymmetricTransmonDesign()
evaluator = AdjointEvaluator(designer, config=base_config)

# (Optional) deterministic sampling for EPR sources across sweeps
evaluator.random_seed = 12345

# Default folder: exp_YYYYMMDD-HH%M-%S (comes from Experiment.__init__)
experiment = Experiment()

x_vals = np.linspace(0.18, 0.21, 8 ,dtype=float)
y_vals = np.linspace(0.24, 0.26, 8 ,dtype=float)
param_grid = np.array([[x, y] for x in x_vals for y in y_vals], dtype=float)

# 5) Optimiser
initial_params = np.array([0.20, 0.20], float)
lr = 0.01
opt = Optimiser(initial_params=initial_params, lr=lr, evaluator=evaluator)

n_adjoint_samples_list = [1, 3, 5, 10, 20, 40, 80, 160]
timestamp = datetime.now().strftime("%Y%m%d-%H%M-%S")

# Save the run-level config once
experiment.save_config({
    "experiment_name": "Pure EPR, differing N adjoint_samples",
    "adjoint_evaluator": evaluator.to_config_dict(),
    "grid": {"x_vals": x_vals.tolist(), "y_vals": y_vals.tolist()},
    "initial_params": initial_params.tolist(),
    "lr": lr,
    "n_adjoint_samples_list": n_adjoint_samples_list,
    "notebook_timestamp": timestamp,
})


for n in n_adjoint_samples_list:
    opt.evaluator.num_adj_sample_points = n
    results = opt.sweep_jj_epr(param_grid, w_jj=0, perturbation_mag=None, verbose=True)
    # File name encodes the weight for easy grep; JSON-serialisation handled by Experiment
    results_filename = f"sweep_pure_epr_{n}_{timestamp}.jsonl"
    experiment.save_results(results, results_filename)
    print("Wrote:", experiment.path(results_filename))
